SMNP

In [164]:
from pysnmp.hlapi import *
from datetime import datetime
import pandas as pd

#snmpwalk -v1 -c public 10.0.1.131 

# Configuración de la consulta SNMP
target = '10.0.1.131'  # Dirección IP del dispositivo SNMP
community = 'public'  # Comunidad SNMP (debe coincidir con la configuración del dispositivo)
port = 161  # Puerto SNMP por defecto

OID_list = ['.1.3.6.1.4.1.29956.3.1.1.1.1.1.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.2.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.3.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.4.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.5.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.6.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.7.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.8.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.9.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.10.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.11.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.12.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.13.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.14.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.15.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.16.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.17.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.18.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.19.1']


sqlite_insert_query = """INSERT INTO freewave_reading
                                                    (date,
                                                    signal,
                                                    noise,
                                                    supply_voltage,
                                                    rx_rate,
                                                    reflected_power,
                                                    temperature,
                                                    range,
                                                    tx_rate,
                                                    sn_delta,
                                                    connected_to,
                                                    upstream_signal,
                                                    upstream_noise,
                                                    disconnect_count,
                                                    packet_rx_count,
                                                    packet_tx_count,
                                                    packet_dropped_count,
                                                    packet_bad_count,
                                                    ID) 

                                            VALUES """ 

In [177]:
reading = []
currentDateAndTime = datetime.now()
date = str(pd.to_datetime(currentDateAndTime))
reading.append(date)

# Realizar una operación SNMP GET
for oid in OID_list:
    errorIndication, errorStatus, errorIndex, varBinds = next(
    getCmd(SnmpEngine(),
           CommunityData(community),
           UdpTransportTarget((target, port)),
           ContextData(),
           ObjectType(ObjectIdentity(oid)))
    )
    # Procesar la respuesta del GET
    if errorIndication:
        print(f"Error: {errorIndication}")
    elif errorStatus:
        print(f"Error en la petición: {errorStatus.prettyPrint()}")
    else:
        for varBind in varBinds:
            reading.append(varBind[1].prettyPrint())
reading

['2024-08-21 16:27:13.803289',
 'No Such Object currently exists at this OID',
 '-69',
 '-111',
 '1193',
 '8470',
 '1',
 '37',
 '7087',
 '0',
 '42',
 'FreeWave',
 '8867258',
 '0',
 '0',
 '2',
 '1369883625',
 '1627494736',
 '37108451',
 '28793']

In [176]:
oid = '.1.3.6.1.4.1.29956.3.1.1.1.1.2.1'  # OID que identifica un objeto SNMP específico

# Realizar una operación SNMP GET

errorIndication, errorStatus, errorIndex, varBinds = next(
    getCmd(SnmpEngine(),
           CommunityData(community),
           UdpTransportTarget((target, port)),
           ContextData(),
           ObjectType(ObjectIdentity(oid)))
)

# Procesar la respuesta del GET
if errorIndication:
    print(f"Error: {errorIndication}")
elif errorStatus:
    print(f"Error en la petición: {errorStatus.prettyPrint()}")
else:
    for varBind in varBinds:
        print(f'GET: {varBind[1].prettyPrint()}')


GET: -69


In [154]:
oid = '1.3.6.1.4.1.29956.3.1.1.1.1.4'

# Realizar una operación SNMP GET-NEXT

# es similar a GET, pero en lugar de devolver el valor de un objeto específico identificado por un OID, 
#devuelve el valor del siguiente objeto en la jerarquía de OID.

errorIndication, errorStatus, errorIndex, varBinds = next(
    nextCmd(SnmpEngine(),
            CommunityData(community),
            UdpTransportTarget((target, port)),
            ContextData(),
            ObjectType(ObjectIdentity(oid)),
            lexicographicMode=False)  # Este parámetro evita recorrer más allá del OID dado
)

# Procesar la respuesta del GET-NEXT
if errorIndication:
    print(f"Error: {errorIndication}")
elif errorStatus:
    print(f"Error en la petición: {errorStatus.prettyPrint()}")
else:
    for varBind in varBinds:
        print(f'GET-NEXT: {varBind.prettyPrint()}')

GET-NEXT: SNMPv2-SMI::enterprises.29956.3.1.1.1.1.4.1 = 1193


In [152]:
oid = '.1.3.6.1.4.1.29956.3.1.1.1.1 .2'

# Realizar una operación SNMP GET-BULK

# GET-BULK es una operación SNMP que permite recuperar un gran número de objetos en una única solicitud, 
# optimizando así el uso de la red cuando necesitas obtener muchos valores de una tabla o una lista de objetos SNMP. 
# Es una mejora sobre las operaciones GET-NEXT y GET en términos de eficiencia, especialmente en redes donde 
# la latencia es un factor importante.

errorIndication, errorStatus, errorIndex, varBinds = next(
    bulkCmd(SnmpEngine(),
            CommunityData(community),
            UdpTransportTarget((target, port)),
            ContextData(),
            0, 25,  # non-repeaters y max-repetitions
            ObjectType(ObjectIdentity(oid)))
)

# Procesar la respuesta del GET-BULK
if errorIndication:
    print(f"Error: {errorIndication}")
elif errorStatus:
    print(f"Error en la petición: {errorStatus.prettyPrint()}")
else:
    for varBind in varBinds:
        #print("varbBid", varBind)
            print("oid", varBind[0])
            print("valor", varBind[1])

oid 1.3.6.1.4.1.29956.3.1.1.1.1.2.1
valor -69


In [53]:
# Realizar una operación SNMP WALK
errorIndication, errorStatus, errorIndex, varBindTable = nextCmd(
    SnmpEngine(),
    CommunityData(community),
    UdpTransportTarget((target, port)),
    ContextData(),
    ObjectType(ObjectIdentity('1.3.6.1.4.1.29956.3.1.1.1.1.4.1')),  # OID base para hacer el WALK (ejemplo: toda la tabla sysDescr)
    lexicographicMode=False  # Para evitar recorrer más allá del prefijo especificado
)



# Procesar la respuesta del WALK
if errorIndication:
    print(f"Error: {errorIndication}")
elif errorStatus:
    print(f"Error en la petición: {errorStatus.prettyPrint()}")
else:
    for varBind in varBindTable:
        for oid, value in varBind:
            print(f'WALK: {oid.prettyPrint()} = {value.prettyPrint()}')


ValueError: not enough values to unpack (expected 4, got 0)